##############################################################################
##############################################################################
### **Atelier "Faire du Machine Learning et du Deep Learning sur des Time Series"**
##############################################################################
##############################################################################
<br>
<br>
<br>
<br>
#### **Partie 3 : Détection d'anomalies sur des séries temporelles et explicabilité sur ces anomalies**

<br>
<br>

Dans ce dernier notebook, nous allons utiliser la méthode de l'isolation forest pour détecter des anomalies sur nos time series. Nous allons aussi utiliser SHAP pour avoir des indices sur les caractéristiques qui expliquent le classement d'un sample en anomalie.

<br>
<br>
<br>
<br>

## Section 0 : récupération, préparation et Fusion des Datasets ##

L'analyse multivariée repose sur l'intégration de variables exogènes (externes) pour améliorer la précision prédictive. Ici, nous fusionnons les données de consommation électrique avec les données météorologiques.

    Alignement temporel : Synchronisation des deux sources sur un index commun via une jointure interne (inner merge).

  

In [ ]:
import requests
import pandas as pd
import time

def fetch_brest_electricity_data():
    base_url = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-metropoles-tr/exports/json"
    
    # Paramètres de la requête
    params = {
        "where": "libelle_metropole='Brest Métropole' AND date_heure >= '2020-01-01' AND date_heure <= '2025-12-31'",
        "order_by": "date_heure ASC",
        "timezone": "UTC"
    }
    
    print("Récupération des données pour Brest Métropole (2020-2025)...")
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        
        data = response.json()
        df = pd.DataFrame(data)
        print(df.head())
            
        df = df[['date_heure', 'consommation']].dropna()
        
        print(f"Chargement Terminé ! {len(df)} lignes récupérées.")
        return df

    except Exception as e:
        print(f"Erreur lors de la récupération : {e}")
        return None


df_conso_brest = fetch_brest_electricity_data()

if df_conso_brest is not None:
    print(df_conso_brest.head())
    # Sauvegarde pour l'atelier
    df_conso_brest.to_csv("conso_brest_2020_2025.csv", index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# 1. indexation temporelle
df_conso_brest = df_conso_brest.rename(columns={"date_heure": "date"})
df_conso_brest['date'] = pd.to_datetime(df_conso_brest['date'], utc=True)
df_conso_brest = df_conso_brest.set_index('date').sort_index()

# changement de temporalité des données
df_conso_brest_journ = df_conso_brest['consommation'].resample('D').mean().interpolate()




In [ ]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Configuration de l'API avec cache et relance automatique en cas d'erreur
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

def get_weather_data_brest(start_date, end_date):
    url = "https://archive-api.open-meteo.com/v1/archive"
    
    params = {
        "latitude": 48.3904, # Brest
        "longitude": -4.4861,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m", "shortwave_radiation"],
        "timezone": "Europe/Berlin"
    }
    
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]

    # Processus de transformation des données horaires
    hourly = response.Hourly()
    hourly_data = {"date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    )}
    
    hourly_data["temp_moy"] = hourly.Variables(0).ValuesAsNumpy()
    hourly_data["humidity"] = hourly.Variables(1).ValuesAsNumpy()
    hourly_data["vent_vitesse"] = hourly.Variables(2).ValuesAsNumpy()
    hourly_data["rayonnement_moyen"] = hourly.Variables(3).ValuesAsNumpy()

    df_meteo = pd.DataFrame(data=hourly_data)
    
    # Passage en format journalier
    df_meteo_journ = df_meteo.resample('D', on='date').mean()
    
    return df_meteo_journ


df_meteo_brest_journ = get_weather_data_brest("2020-01-01", "2025-12-31")
print(df_meteo_brest_journ.head())

In [ ]:
# 2. Fusion (Jointure sur la date)

df_data_multi_brest = pd.merge(df_conso_brest_journ, df_meteo_brest_journ, on='date', how='inner')

print(df_data_multi_brest.head())



## Section 1 : Ingénierie des caractéristiques ##

Nous allons aider le modèle en construisant des caractéristiques reflétant la position du chargé de prédiction le jour j.


In [ ]:
# --- PASSÉ (Lags) ---
# Consommation de la veille (J-1), de l'avant veille (J-2) et de la semaine dernière (J-7) pour la saisonnalité
df_data_multi_brest['conso_obs_j-1'] = df_data_multi_brest['consommation'].shift(1)
df_data_multi_brest['conso_obs_j-2'] = df_data_multi_brest['consommation'].shift(2)
df_data_multi_brest['conso_obs_j-7'] = df_data_multi_brest['consommation'].shift(7)

# Météo observée hier (J-1) pour l'inertie thermique
df_data_multi_brest['temp_obs_j-1'] = df_data_multi_brest['temp_moy'].shift(1)

# Météo observée hier (J-1) pour l'humidité
df_data_multi_brest['humidity_j-1'] = df_data_multi_brest['humidity'].shift(1)

# Météo observée hier (J-1) pour le rayonnement
df_data_multi_brest['rayonnement_moyen_j-1'] = df_data_multi_brest['rayonnement_moyen'].shift(1)

# Météo observée hier (J-1) pour le vent
df_data_multi_brest['vent_vitesse_j-1'] = df_data_multi_brest['vent_vitesse'].shift(1)

# --- FUTUR / PRÉVISIONS (Leads) ---
# On utilise la donnée réelle de J comme si c'était la prévision faite le matin même
df_data_multi_brest['temp_prev_j'] = df_data_multi_brest['temp_moy'] 

# On utilise la donnée de J+1 comme prévision pour demain (Lead)
df_data_multi_brest['temp_prev_j+1'] = df_data_multi_brest['temp_moy'].shift(-1)


df_data_multi_brest = df_data_multi_brest.dropna()

print(df_data_multi_brest.head())


# Définition des périodes
dataset_train = df_data_multi_brest['2015-01-01':'2024-12-31']
dataset_test = df_data_multi_brest['2025-01-01':'2025-12-31']




In [ ]:
import holidays

# On définit les jours fériés en France 
fr_holidays = holidays.France()

def add_calendar_features(df):
    # On travaille sur une copie pour ne pas modifier l'original par mégarde
    df_enriched = df.copy()
    
    # 1. Variables temporelles basiques
    df_enriched['day_of_week'] = df_enriched.index.dayofweek
    df_enriched['month'] = df_enriched.index.month
    df_enriched['is_weekend'] = df_enriched.index.dayofweek.isin([5, 6]).astype(int)
    
    # 2. Jours fériés (Boolean : 1 si férié, 0 sinon)
    df_enriched['is_holiday'] = df_enriched.index.map(lambda x: 1 if x in fr_holidays else 0)
    
    # 3.  Veille et Lendemain de jour férié 
    df_enriched['is_holiday_prev'] = df_enriched['is_holiday'].shift(-1, fill_value=0)
    df_enriched['is_holiday_next'] = df_enriched['is_holiday'].shift(1, fill_value=0)
    
    return df_enriched

# Application sur vos datasets
dataset_train_enriched = add_calendar_features(dataset_train)
dataset_test_enriched = add_calendar_features(dataset_test)

print(dataset_test_enriched.head())

## Section 2 : Détection des anomalies sur la consommation électrique de 2025 à Brest ##

Nous allons maintenant utiliser la méthode de la forêt isolée pour détecter les anomalies de consommation.
<br><br>
L'idée sous-jacente de cette approche est de mesurer pour chaque échantillon combien de profondeur d'arbre de décision faut-il pour l'isoler de toute le reste du dataset. 
<br><br>
L'idée : si cette profondeur est faible, ce sample est facilement séparable, il est une anomalie

In [ ]:
from sklearn.ensemble import IsolationForest
import pandas as pd

# 1. Nettoyage des données
train_clean = dataset_train_enriched.dropna()
test_clean = dataset_test_enriched.dropna()

# 2. Initialisation du modèle
# n_estimators : nombre d'arbres 
# contamination : proportion estimée d'anomalies, sert à choisir le treshold détectant les anomalies 
model_if = IsolationForest(n_estimators=100, 
                           contamination=0.01)

# 3. Entraînement de la foret sur le dataset d'entrainement
#######################################
#ligne à remplir
#######################################
model_if.fit()

# 4. prédictions
#######################################
#ligne à remplir
#######################################
predictions = model_if.predict()


test_clean['is_anomaly'] = predictions

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 6))
# Tracer la consommation réelle
plt.plot(test_clean.index, test_clean['consommation'], color='blue', alpha=0.4, label='Consommation')

# Identifier les anomalies
#######################################
#ligne à remplir
#######################################
anomalies = test_clean[]

# Superposer les points rouges
plt.scatter(anomalies.index, anomalies['consommation'], color='red', label='Anomalie Détectée')

plt.title('Détection d\'événements rares via Isolation Forest')
plt.legend()
plt.show()

<br><br>

Nous avons bien détecté des anomalies en 2025. A vous de jouer, explorer ces anomalies et comprenez à quels jours particuliers de l'année sont-ils associés.

<br><br>

Cette analyse est intéressante, car nous comprendons rapidement la rareté évènementielle, mais qu'en est-il des paramètres qui expliquent ces anomalies ? est-ce des facteurs météo ou bien des facteurs calendaires ?

<br><br>

## Section 3 d'explicabilité : Montrer pourquoi une anomalie est une anomalie ##

In [ ]:
import shap
import numpy as np

# On s'assure que les données passées à SHAP sont des valeurs pures (numpy) 
# pour éviter les conflits de noms de colonnes ou d'index de Pandas
features_cols = train_clean.columns.tolist()
X_reference = train_clean[features_cols].values
X_anomaly = test_clean[test_clean['is_anomaly'] == -1][features_cols].values
print(test_clean[test_clean['is_anomaly'] == -1][features_cols])
print(X_anomaly[0:1])


# 1. Initialisation de l'explainer avec le modèle déjà entraîné
# On utilise un échantillon du train pour la baseline
explainer = shap.TreeExplainer(model_if, data=X_reference[:100])

# 2. Calcul des SHAP values pour la première anomalie détectée
#######################################
#ligne à remplir
#######################################
shap_values_single = explainer.shap_values()

# 3. Visualisation
shap.force_plot(
    explainer.expected_value, 
    shap_values_single[0], 
    features=X_anomaly[0:1],
    #######################################
    #ligne à remplir
    ####################################### 
    feature_names=,
    matplotlib=True
)



<br><br>
La visualisation n'est pas satisfaisante, on va plutot utiliser l'affichage en cascade
<br><br>

In [ ]:

explainer = shap.Explainer(model_if, train_clean)
shap_values = explainer(test_clean[features_cols])

# 2. On sélectionne l'index d'une anomalie détectée (ex: la première)
#######################################
#ligne à remplir
#######################################
anomalie_idx = test_clean[].index[0]
# On récupère sa position entière dans le tableau pour SHAP
loc_idx = test_clean.index.get_loc(anomalie_idx)

# 3. Affichage en "Waterfall" (Cascade)
plt.figure(figsize=(10, 8))
shap.plots.waterfall(shap_values[loc_idx], max_display=15)

<br><br>
A vous de jouer, expliquer pour chaque anomalie quels sont les facteurs prépondérants expliquant la surconsommation ou la sous-consommation. Est-ce cohérent à votre avis ?
<br><br>